<a href="https://colab.research.google.com/github/ImSittingOnSalt/Pizza-or-not-/blob/main/ItWorks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader , random_split
from torchvision import datasets, transforms
import torchvision.models as model
import random
import numpy as np
from PIL import Image
import os
import pathlib
import matplotlib.pyplot as plt
from os import listdir


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/My Drive/pizza_not_pizza/pizza/'
images = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(".jpg") or file_name.endswith(".png"):
        img = Image.open(os.path.join(folder_path, file_name))
        images.append(img)

print( images[0], images[235], images[747], images[43])

Mounted at /content/drive
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x385 at 0x7CD2A5AEAB00> <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x341 at 0x7CD2A587D810> <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x382 at 0x7CD2A56B6920> <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7CD2A59C49A0>


In [ ]:
# Получаю имена классов
data_dir = pathlib.Path('/content/drive/My Drive/pizza_not_pizza/')
class_names = [item.name for item in data_dir.glob('*')][:2]
print(class_names)

['pizza', 'not_pizza']


In [ ]:
pizza_dir = '/content/drive/My Drive/pizza_not_pizza/pizza/'         #дирректории классов
not_pizza_dir = '/content/drive/My Drive/pizza_not_pizza/not_pizza/'

In [ ]:
len_pizza_img = len(os.listdir(pizza_dir))          #сколько фотов пицы......
len_not_pizza_img = len(os.listdir(not_pizza_dir))
print(len_pizza_img, len_not_pizza_img)

983 983


# **Visualizing images**

In [ ]:
# plt.imshow(images[301])
# plt.show()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Разделение на тренировочный и валидационный наборы
train_size = int(0.7 * len(dataset))  # 70% для тренировки
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

# Define data loaders with batch size
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
# Рисую рандомные картинки
def plot_random_images():
    images, labels = next(iter(train_loader))  # Беру батч изображений с классами

    plt.figure(figsize=(10, 5))

    for i, (image, label) in enumerate(zip(images, labels)):
        plt.subplot(4, 5, i + 1)
        plt.imshow(image.permute(1, 2, 0))  # Переключить отображение в порядке RGB
        plt.title(class_names[label])
        plt.axis('off')
        if i == 20:
            break

    plt.tight_layout()
    plt.show()

In [ ]:
# plot_random_images() #выдаёт ошибку, не понимаю в чём мем, но картинки всё равно рисует :)

In [ ]:
class ImprovedModel(nn.Module):
    def __init__(self):
        super(ImprovedModel, self).__init__()

        # Определение сверточных слоев здесь
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=3, stride=1, padding=1), # 2D сверточный слой с: Входными каналами,Выходными каналами, Размером ядра: 3x3, Шагом: 1, Отступом: 1
            nn.BatchNorm2d(24),                                   # Нормализация по батчам.
            nn.ReLU(),                                            # Функция активации
            nn.MaxPool2d(kernel_size=2, stride=2),                # Макс. пулинг для понижения дискретизации и уменьшения пространственной размерности.

            nn.Conv2d(24, 40, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(40, 80, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU(),

            nn.Conv2d(80, 160, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU(),

            nn.Conv2d(160, 160, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU(),

            nn.Conv2d(160, 320, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(320),
            nn.ReLU(),

            nn.Conv2d(320, 320, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(320),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)) #Адаптивный пулинг по среднему значению, автоматически подстраивающийся под размер входных данных
        )

        # Определение полностью связанных слоев здесь
        self.fc_layers = nn.Sequential(
            nn.Flatten(),         # Преобразует выход из сверточных слоев в 1D вектор
            nn.Linear(320, 128),
            nn.ReLU(),
            nn.Linear(128, 42),
            nn.ReLU(),
            nn.Linear(42, 1),     # Выходной слой с 1 единицей (бинарная классификация)
            nn.Sigmoid()          # Для получения вероятности принадлежности к одному из классов (0 или 1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [ ]:
def train_and_evaluate_model(train_loader, val_loader, epo, mdl=None): # rate was taken empirically. This value is optimal in the base
    print(f"Train size:\t\t{len(train_loader.dataset)}\n"
          f"Validation size:\t{len(val_loader.dataset)}\n")


    model = mdl if mdl is not None else ImprovedModel()
    loss_fn = torch.nn.CrossEntropyLoss()
    opt = torch.optim.Adam(model.parameters(), lr=1e-5)

    history = []
    for epoch in range(epo):
      model.train()  # Установить режим обучения для модели
      train_loss = 0.0
      for data, target in train_loader:
        target = target.unsqueeze(1)  # Добавить измерение для соответствия выходному размеру
        target = target.clone().detach().float()
        # Прямой проход (forward pass)
        output = model(data)

        # Вычислить ошибку (loss)
        loss = loss_fn(output, target)
        # Обратный проход (backward pass) и обновление весов
        opt.zero_grad()
        loss.backward()
        opt.step()
        train_loss += loss.item()

      model.eval()  # Установить режим оценки для модели
      with torch.no_grad():  # Отключить вычисление градиентов для валидации
        val_loss = 0.0
        val_acc = 0.0
        for data, target in val_loader:
          output = model(data)
          val_loss += loss_fn(output, target).item()
          val_acc += (output.round() == target).sum().item()
        val_acc /= len(val_loader.dataset)

      # Сохранить информацию об эпохе в history
      history.append((train_loss / len(train_loader), val_loss / len(val_loader), val_acc))

      print(f"Epoch: {epoch+1}/{epo}\tTrain Loss: {train_loss:.3f}\tVal Loss: {val_loss:.3f}\tVal Acc: {val_acc:.3f}")

    return val_acc, history

def outputs(model_type, text, plot):
    print(f"{model_type} Accuracy: {text:.3f}\n")
    plt.plot_history(plot, "loss")

In [ ]:
accuracy, history = train_and_evaluate_model(train_loader, val_loader, epo=25)  # сейчас споткнулась тут, не получается, думаю

Train size:		1376
Validation size:	590



RuntimeError: expected scalar type Long but found Float